In [1]:
import iris
import os
import shutil
import dask

In [2]:
dirs = ['3hrly',  '6hrly',  'daily'  ,'hourly' ,'monthly']
for dir in dirs:
    print(dir)
    ! ls /data/cssp-china/mini-dataset-24-01-19/20CR/{dir} | wc -l
    ! ls /data/cssp-china/mini-dataset-24-01-19/nc/{dir} | wc -l
    print('\n')

3hrly
5850
5850


6hrly
5850
1447


daily
5850
5850


hourly
5850
5850


monthly
1920
1920




In [2]:
SUB_DIR = '6hrly'

In [3]:
SAMPLE_DATA_PATH=f"/data/cssp-china/mini-dataset-24-01-19/20CR/{SUB_DIR}"

In [4]:
!ls {SAMPLE_DATA_PATH} | head

apepda.pb51150.pp
apepda.pb511f0.pp
apepda.pb511p0.pp
apepda.pb51240.pp
apepda.pb512e0.pp
apepda.pb512o0.pp
apepda.pb51360.pp
apepda.pb513g0.pp
apepda.pb513q0.pp
apepda.pb51450.pp
ls: write error: Broken pipe


In [5]:
files = [os.path.join(SAMPLE_DATA_PATH, f) for f in sorted(os.listdir(SAMPLE_DATA_PATH)) if f[-3:]=='.pp' ]

In [6]:
eg_path = files[3]

In [7]:
IN_DIR =  f"/data/cssp-china/mini-dataset-24-01-19/20CR/{SUB_DIR}"
OUT_DIR = f"/data/cssp-china/mini-dataset-24-01-19/nc/{SUB_DIR}"
!mkdir -p {OUT_DIR}

In [8]:
from dask import bag as db

In [9]:
from collections import namedtuple
Paths = namedtuple('Paths',['old','new'])
def get_new_and_old_path(path):
    filename = os.path.basename(path)
    new_filename = filename[:-2] + 'nc' if filename[-2:].lower() == 'pp' else filename + '.nc'
    new_path = os.path.join(OUT_DIR, new_filename)
    return Paths(path, new_path)

eg_path = get_new_and_old_path(files[0] )
eg_path

Paths(old='/data/cssp-china/mini-dataset-24-01-19/20CR/6hrly/apepda.pb51150.pp', new='/data/cssp-china/mini-dataset-24-01-19/nc/6hrly/apepda.pb51150.nc')

In [10]:
def not_converted(paths):
    return not os.path.exists(paths.new)

not_converted(eg_path)

False

In [11]:
paths = eg_path

In [12]:
paths

Paths(old='/data/cssp-china/mini-dataset-24-01-19/20CR/6hrly/apepda.pb51150.pp', new='/data/cssp-china/mini-dataset-24-01-19/nc/6hrly/apepda.pb51150.nc')

In [13]:
# import shutil
# import dask
# def local_convert(paths):
#     tmp_path = '/tmp/'+os.path.basename(paths.old)
#     tmp_path_new = '/tmp/'+os.path.basename(paths.new)
#     shutil.copy(paths.old, tmp_path)
    
#     with dask.config.set(scheduler='single-threaded'):
#         precipitation_flux
#         iris.fileformats.netcdf.save(cube, tmp_path_new, zlib=True, complevel=2)
    
    
#     shutil.copy(tmp_path_new, paths.new)
    
#     try:
#         os.remove(tmp_path_new)
#     except Exception:
#         print(f"could not delete {tmp_path_new}")
              
#     try:
#         os.remove(tmp_path)
#     except Exception:
#         print(f"could not delete {tmp_path}")

#     return paths

In [14]:
from pathlib import Path
def convert(paths):
    with dask.config.set(scheduler='single-threaded'):
        cubes = iris.load(paths.old)
        Path(os.path.dirname(paths.new)).mkdir(parents=True, exist_ok=True)
        iris.fileformats.netcdf.save(cubes, paths.new, zlib=False)
        return paths

In [51]:
%%time
convert(eg_path)

CPU times: user 7.99 s, sys: 1.27 s, total: 9.26 s
Wall time: 14.9 s


Paths(old='/data/cssp-china/mini-dataset-24-01-19/20CR/6hrly/apepda.pb51150.pp', new='/data/cssp-china/mini-dataset-24-01-19/nc/6hrly/apepda.pb51150.nc')

In [15]:
! ls -lh {eg_path.new} {eg_path.old}

-rwxrwxrwx 1 root root 149M Jan 26 15:15 /data/cssp-china/mini-dataset-24-01-19/20CR/6hrly/apepda.pb51150.pp
-rwxrwxrwx 1 root root 312M Feb 10 13:27 /data/cssp-china/mini-dataset-24-01-19/nc/6hrly/apepda.pb51150.nc


In [16]:
import distributed
import dask
from dask_kubernetes import KubeCluster
from dask import bag as db

from dask.diagnostics import ProgressBar

from functools import partial



In [20]:
conversion = db.from_sequence(files).map(get_new_and_old_path).filter(not_converted).map(convert)
conversion

dask.bag<convert..., npartitions=101>

In [18]:
cluster = KubeCluster()
cluster.scale_up(30)
display(cluster)
# Connect dask to the cluster
client = distributed.Client(cluster)
client

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.244.0.48:40469
distributed.scheduler - INFO -   dashboard at:                     :8787


distributed.scheduler - INFO - Receive client connection: Client-9e73befa-4cbd-11ea-80cf-3f98550bcf77
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.244.0.48:40469 Dashboard: /user/tam203/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register tcp://10.244.0.50:43345
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.50:43345
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.244.0.49:44239
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.49:44239
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.244.0.51:46857
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.51:46857
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.244.9.27:34877
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.9.27:34877
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.244.9.26:43517
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.9.26:43517
distributed.c

In [ ]:
to_convert = db.from_sequence(files).map(get_new_and_old_path).filter(not_converted).to_converte()
to_convert

In [22]:
%%time
with ProgressBar():
    converted = conversion.compute()
converted

distributed.scheduler - INFO - Remove worker tcp://10.244.9.32:46103
distributed.core - INFO - Removing comms to tcp://10.244.9.32:46103
distributed.scheduler - INFO - Remove worker tcp://10.244.9.26:43517
distributed.core - INFO - Removing comms to tcp://10.244.9.26:43517
distributed.scheduler - INFO - Remove worker tcp://10.244.9.30:33875
distributed.core - INFO - Removing comms to tcp://10.244.9.30:33875
distributed.scheduler - INFO - Remove worker tcp://10.244.9.33:46633
distributed.core - INFO - Removing comms to tcp://10.244.9.33:46633
distributed.scheduler - INFO - Remove worker tcp://10.244.9.29:44169
distributed.core - INFO - Removing comms to tcp://10.244.9.29:44169
distributed.batched - INFO - Batched Comm Closed: in <closed TCP>: Stream is closed
distributed.scheduler - INFO - Remove worker tcp://10.244.9.28:41663
distributed.core - INFO - Removing comms to tcp://10.244.9.28:41663
distributed.scheduler - INFO - Remove worker tcp://10.244.0.49:44239
distributed.core - INFO -

CPU times: user 4.3 s, sys: 565 ms, total: 4.86 s
Wall time: 1min 35s


[Paths(old='/data/cssp-china/mini-dataset-24-01-19/20CR/6hrly/apepda.pbg3a10.pp', new='/data/cssp-china/mini-dataset-24-01-19/nc/6hrly/apepda.pbg3a10.nc')]

In [ ]:
cluster.close()